Import the right libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import json
import pycountry
from sklearn.linear_model import LinearRegression

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models.layouts import TabPanel, Tabs
from bokeh.layouts import gridplot, column
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, LinearAxis, FactorRange, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer


Output in jupyter notebook

In [2]:
output_notebook()

Loading BokehJS ...

Opening and preprocessing the datasets

Concatting the crashes and set the date format for the date

Crashes

In [3]:
csv_files_crashes = ['data/stats_crashes_202106_overview.csv', 
                     'data/stats_crashes_202107_overview.csv', 
                     'data/stats_crashes_202108_overview.csv',
                     'data/stats_crashes_202109_overview.csv',
                     'data/stats_crashes_202110_overview.csv',
                     'data/stats_crashes_202111_overview.csv',
                     'data/stats_crashes_202112_overview.csv']


dfs_crashes = [pd.read_csv(file, encoding='utf-16') for file in csv_files_crashes]
df_crashes = pd.concat(dfs_crashes, ignore_index=True)
df_crashes['Date'] = pd.to_datetime(df_crashes["Date"], format='%Y-%m-%d')
df_crashes['Month'] = df_crashes['Date'].dt.month
df_crashes

,Date,Package Name,Daily Crashes,Daily ANRs,Month
0,2021-06-01,com.vansteinengroentjes.apps.ddfive,15,1,6
1,2021-06-02,com.vansteinengroentjes.apps.ddfive,12,1,6
2,2021-06-03,com.vansteinengroentjes.apps.ddfive,20,1,6
3,2021-06-04,com.vansteinengroentjes.apps.ddfive,13,0,6
4,2021-06-05,com.vansteinengroentjes.apps.ddfive,14,0,6
...,...,...,...,...,...
209,2021-12-27,com.vansteinengroentjes.apps.ddfive,64,0,12
210,2021-12-28,com.vansteinengroentjes.apps.ddfive,60,0,12
211,2021-12-29,com.vansteinengroentjes.apps.ddfive,37,0,12
212,2021-12-30,com.vansteinengroentjes.apps.ddfive,46,1,12


Sales

In [4]:

csv_files_sales_1 = ['data/sales_202106.csv',
                   'data/sales_202107.csv',
                   'data/sales_202108.csv',
                   'data/sales_202109.csv',
                   'data/sales_202110.csv',]

csv_files_sales_2 = ['data/sales_202111.csv',
                   'data/sales_202112.csv']

# Convert to DataFrame
dfs_sales_1 = [pd.read_csv(file, encoding='utf-8') for file in csv_files_sales_1]
dfs_sales_2 = [pd.read_csv(file, encoding='utf-8') for file in csv_files_sales_2]

# Concatenate all DataFrames into one, resetting the index
df_sales_1 = pd.concat(dfs_sales_1, ignore_index=True)
df_sales_2 = pd.concat(dfs_sales_2, ignore_index=True)

# Convert 'Transaction Date' to pd datetime
df_sales_Conversion = df_sales_1.groupby('Buyer Currency')["Currency Conversion Rate"].mean()
df_sales_1['Transaction Date'] = pd.to_datetime(df_sales_1['Transaction Date'], format='%b %d, %Y')
df_sales_2['Order Charged Date'] = pd.to_datetime(df_sales_2['Order Charged Date'], format='%Y-%m-%d')
df_sales_2 = df_sales_2.merge(df_sales_Conversion, left_on='Currency of Sale', right_on='Buyer Currency')

df_sales_2['Charged Amount'] = df_sales_2['Charged Amount'].astype(float).round(2)
df_sales_1['Amount (Buyer Currency)'] = df_sales_1['Amount (Buyer Currency)'].astype(float).round(2)
df_sales_2['Amount (Merchant Currency)'] = df_sales_2['Charged Amount'].multiply(df_sales_2['Currency Conversion Rate'], axis=0).round(2)
 
# Rename columns to match
df_sales_1 = df_sales_1.rename(columns={'Description': 'Order Number',   
                                        'Transaction Date': 'Order Charged Date',
                                        'Transaction Type': 'Financial Status',
                                        'Product id': 'Product ID',
                                        'Sku Id': 'SKU ID',
                                        'Buyer Currency': 'Currency of Sale',
                                        'Buyer Country': 'Country of Buyer',
                                        'Buyer Postal Code': 'Postal Code of Buyer',
                                        'Amount (Buyer Currency)': 'Charged Amount'})

columns = ['Order Number', 'Order Charged Date', 'Financial Status', 
           'Product ID', 'Product Title', 'SKU ID', 'Country of Buyer',
           'Postal Code of Buyer', 'Charged Amount', 'Currency of Sale', 
           "Currency Conversion Rate", 'Amount (Merchant Currency)']

# Concatenate both DataFrames
df_sales = pd.concat([df_sales_1[columns], df_sales_2[columns]], ignore_index=True)

df_sales['Charged Amount'] = df_sales["Charged Amount"].astype(str).str.replace(',','')
df_sales['Charged Amount'] = pd.to_numeric(df_sales['Charged Amount'])
# Filter the DataFrame
df_sales = df_sales[
    ((df_sales['Financial Status'] == 'Charge') | (df_sales['Financial Status'] == 'Charged')) &
    (df_sales['Product ID'] == 'com.vansteinengroentjes.apps.ddfive')
      ]


df_sales['Year'] = df_sales['Order Charged Date'].dt.year
df_sales['Month'] = df_sales['Order Charged Date'].dt.month
df_sales



,Order Number,Order Charged Date,Financial Status,Product ID,Product Title,SKU ID,Country of Buyer,Postal Code of Buyer,Charged Amount,Currency of Sale,Currency Conversion Rate,Amount (Merchant Currency),Year,Month
4,GPA.3370-7096-7934-01916,2021-06-01,Charge,com.vansteinengroentjes.apps.ddfive,Character Manager (Complete Reference for DnD 5),unlockcharactermanager,US,62011,5.49,USD,0.818700,4.49,2021,6
6,GPA.3301-2849-0660-49349,2021-06-01,Charge,com.vansteinengroentjes.apps.ddfive,DM Tools (Complete Reference for DnD 5),premium,US,55320,3.49,USD,0.818250,2.86,2021,6
8,GPA.3372-1497-1097-13226,2021-06-02,Charge,com.vansteinengroentjes.apps.ddfive,Character Manager (Complete Reference for DnD 5),unlockcharactermanager,US,54220,5.49,USD,0.820650,4.51,2021,6
10,GPA.3397-6490-8608-67650,2021-06-02,Charge,com.vansteinengroentjes.apps.ddfive,Character Manager (Complete Reference for DnD 5),unlockcharactermanager,US,78250,5.49,USD,0.819250,4.50,2021,6
12,GPA.3378-4840-7906-77859,2021-06-02,Charge,com.vansteinengroentjes.apps.ddfive,DM Tools (Complete Reference for DnD 5),premium,US,74830,3.49,USD,0.818750,2.86,2021,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,GPA.3358-8615-1171-80349,2021-12-31,Charged,com.vansteinengroentjes.apps.ddfive,DM Tools (Complete Reference for DnD 5),premium,US,98856,3.80,USD,0.847129,3.22,2021,12
3475,GPA.3397-7418-5342-99790,2021-12-31,Charged,com.vansteinengroentjes.apps.ddfive,Character Manager (Complete Reference for DnD 5),unlockcharactermanager,US,98856,5.97,USD,0.847129,5.06,2021,12
3476,GPA.3333-5146-4957-35294,2021-12-31,Charged,com.vansteinengroentjes.apps.ddfive,DM Tools (Complete Reference for DnD 5),premium,GB,NaN,2.59,GBP,1.170363,3.03,2021,12
3477,GPA.3306-0097-9714-42420,2021-12-31,Charged,com.vansteinengroentjes.apps.ddfive,DM Tools (Complete Reference for DnD 5),premium,US,83401,3.49,USD,0.847129,2.96,2021,12


Ratings

In [5]:

csv_files_ratings = ['data/stats_ratings_202106_country.csv',
                        'data/stats_ratings_202107_country.csv',
                        'data/stats_ratings_202108_country.csv',
                        'data/stats_ratings_202109_country.csv',
                        'data/stats_ratings_202110_country.csv',
                        'data/stats_ratings_202111_country.csv',
                        'data/stats_ratings_202112_country.csv']



dfs_ratings = [pd.read_csv(file, encoding='utf-16') for file in csv_files_ratings]

df_ratings = pd.concat(dfs_ratings, ignore_index=True)

df_ratings['Date'] = pd.to_datetime(df_ratings["Date"], format='%Y-%m-%d')
df_ratings['Month'] = df_ratings['Date'].dt.month

df_ratings


,Date,Package Name,Country,Daily Average Rating,Total Average Rating,Month
0,2021-06-01,com.vansteinengroentjes.apps.ddfive,AR,NaN,4.20,6
1,2021-06-01,com.vansteinengroentjes.apps.ddfive,AT,NaN,3.91,6
2,2021-06-01,com.vansteinengroentjes.apps.ddfive,AU,NaN,4.19,6
3,2021-06-01,com.vansteinengroentjes.apps.ddfive,BA,NaN,5.00,6
4,2021-06-01,com.vansteinengroentjes.apps.ddfive,BD,NaN,5.00,6
...,...,...,...,...,...,...
18612,2021-12-31,com.vansteinengroentjes.apps.ddfive,US,NaN,4.04,12
18613,2021-12-31,com.vansteinengroentjes.apps.ddfive,UY,NaN,4.86,12
18614,2021-12-31,com.vansteinengroentjes.apps.ddfive,VE,NaN,4.00,12
18615,2021-12-31,com.vansteinengroentjes.apps.ddfive,VN,NaN,5.00,12


[10p] Sales Volume: Visualize the sales over time (for example, per month or per day) in 
terms of at least two measures. For example: real money (Amount) and transaction count 
(row count). 

In [6]:
# df_sales_monthly = df_sales[['Order Charged Date', 'Charged Amount']].groupby(pd.Grouper(key='Order Charged Date', freq="ME")).sum()
df_sales_monthly = df_sales.groupby('Month')[['Amount (Merchant Currency)']].sum()
monthly_sales_fig = figure(background_fill_color='white',
             border_fill_color='white',
             height=300,
             width=500,
             x_axis_label='Months',
            #  x_axis_type='datetime',
             y_axis_label='Sales',
             y_axis_location='left',
             title='Sales',
             title_location='right',
             toolbar_location='below',
             tools='save')


monthly_sales_fig.step('Month', 'Amount (Merchant Currency)',
         color='Red', source=df_sales_monthly)

show(monthly_sales_fig)
df_sales_monthly


,Amount (Merchant Currency)
Month,
6,857.24
7,1078.06
8,979.57
9,903.84
10,860.16
11,962.25
12,954.24


[15p] Attribute Segmentation and Filtering: Present sales volume (as above) segmented per 
attribute: at least the SKU id (in-app purchase option) attribute should be included, but you 
can also think of the day of the week, time of the day or the country of the customer. 

In [7]:
df_sales_premium = df_sales.loc[df_sales['SKU ID'] == 'premium'].groupby(['Month'])[['Amount (Merchant Currency)']].sum().reset_index()
df_sales_ucm = df_sales.loc[df_sales['SKU ID'] == 'unlockcharactermanager'].groupby(['Month'])[['Amount (Merchant Currency)']].sum().reset_index()

sales_by_sku_fig = figure(background_fill_color='white',
             border_fill_color='white',
             height=300,
             width=500,
             x_axis_label='Months',
            #  x_axis_type='datetime',
             y_axis_label='Sales',
             y_axis_location='left',
             title='Sales',
             title_location='right',
             toolbar_location='below',
             tools='save')

sales_by_sku_fig.step('Month', 'Amount (Merchant Currency)',
         color='Red', source=df_sales_premium)

sales_by_sku_fig.step('Month', 'Amount (Merchant Currency)',
         color='Blue', source=df_sales_ucm)

show(sales_by_sku_fig)

In [8]:
df_sales_by_country = df_sales.groupby(['Country of Buyer'])[['Amount (Merchant Currency)']].sum().reset_index()
df_sales_by_country = df_sales_by_country.sort_values(['Amount (Merchant Currency)'], ascending=False)
sales_by_country_fig = figure(background_fill_color='white',
             border_fill_color='white',
             height=300,
             width=500,
             x_axis_label='Country',
             x_range=df_sales_by_country['Country of Buyer'].head(10),
             y_axis_label='Sales',
             y_axis_location='left',
             title='Sales by country',
             title_location='right',
             toolbar_location='below',
             tools='save')

sales_by_country_fig.vbar(x='Country of Buyer', top='Amount (Merchant Currency)', 
         color='Red', source=df_sales_by_country.head(10))

show(sales_by_country_fig)


[15p] Ratings vs Stability: Can you come up with some Key Performance Indicators (metrics 
and scores) that help management understand how the app is doing in terms of stability and 
user satisfaction? Visualize them in a nice way. For example, the number of crashes in 
correlation with the daily average rating. 

In [9]:
# Convert Date column to string to avoid Bokeh validation issues
df_crashes['Date'] = df_crashes['Date'].astype(str)
df_ratings['Date'] = df_ratings['Date'].astype(str)

# Merge datasets on Date
merged_df = pd.merge(df_crashes, df_ratings, on='Date', how='inner')
merged_df = merged_df.dropna(subset=['Daily Crashes', 'Daily Average Rating'])

avg_crashes = merged_df['Daily Crashes'].mean()
avg_anrs = merged_df['Daily ANRs'].mean()

correlation = merged_df[['Daily Crashes', 'Daily Average Rating']].corr().iloc[0, 1]
X = merged_df[['Daily Crashes']]
y = merged_df['Daily Average Rating']
reg = LinearRegression().fit(X, y)
merged_df['Regression Line'] = reg.predict(X)

# Stability Score (Inverse relation of crashes to rating)
merged_df['Stability Score'] = merged_df['Daily Average Rating'] / (1 + merged_df['Daily Crashes'])

sorted_df = merged_df.sort_values(by='Daily Average Rating')

source = ColumnDataSource(merged_df)
regression_source = ColumnDataSource(sorted_df)

# Create Bokeh figure
p1 = figure(title=f'Correlation between Crashes and Ratings: {correlation:.2f}',
           x_axis_label='Daily Crashes', y_axis_label='Daily Average Rating',
           tools='pan,wheel_zoom,box_zoom,reset,save', width=800, height=400)

# Scatter plot for data points
p1.scatter('Daily Crashes', 'Daily Average Rating', source=source, size=8, color='navy', alpha=0.6, legend_label="Data Points")

# Trend line (Regression Line)
p1.line('Daily Crashes', 'Regression Line', source=regression_source, line_width=2, color='red', legend_label="Trend Line")


# Prepare data for Bokeh
unique_dates = merged_df['Date'].unique().tolist()

# Prepare data for Bokeh
# source = ColumnDataSource(merged_df)

# # Scatter plot for Crashes vs Ratings
# p1 = figure(title=f'Correlation between Crashes and Ratings: {correlation:.2f}', x_axis_label='Daily Crashes', y_axis_label='Daily Average Rating', tools='pan,wheel_zoom,box_zoom,reset,save', width=800, height=400)
# p1.scatter('Daily Crashes', 'Daily Average Rating', source=source, size=8, color='navy', alpha=0.6)

# Line plot for Stability Score Over Time
p2 = figure(title='Stability Score Over Time', x_axis_label='Date', y_axis_label='Stability Score', x_range=FactorRange(*unique_dates), tools='pan,wheel_zoom,box_zoom,reset,save', width=800, height=400)
p2.line('Date', 'Stability Score', source=source, line_width=2, color='green')
p2.scatter('Date', 'Stability Score', source=source, size=6, color='red')
p2.xaxis.major_label_orientation = 1.2

# Show plots
show(column(p1, p2))

# Print KPIs
print(f'Average Daily Crashes: {avg_crashes:.2f}')
print(f'Average Daily ANRs: {avg_anrs:.2f}')
print(f'Correlation between Crashes and Ratings: {correlation:.2f}')

Average Daily Crashes: 21.99
Average Daily ANRs: 0.35
Correlation between Crashes and Ratings: -0.05


In [10]:
df_ratings_by_month = df_ratings.groupby('Month')[['Total Average Rating']].mean().reset_index()

ratings_by_month_fig = figure(background_fill_color='white',
             border_fill_color='white',
             height=300,
             width=500,
             x_axis_label='Month',
             y_axis_label='Average Rating',
             y_axis_location='left',
             title='Sales by country',
             title_location='right',
             toolbar_location='below',
             tools='save')

ratings_by_month_fig.step('Month', 'Total Average Rating',
         color='Red', source=df_ratings_by_month)


show(ratings_by_month_fig)

In [11]:
df_crashes_by_month = df_crashes.groupby(['Month'])[['Daily Crashes']].sum().reset_index()
crashes_by_month_fig = figure(background_fill_color='white',
             border_fill_color='white',
             height=300,
             width=500,
             x_axis_label='Month',
             y_axis_label='Daily Crashes',
             y_axis_location='left',
             title='Sales by country',
             title_location='right',
             toolbar_location='below',
             tools='save')

crashes_by_month_fig.step('Month', 'Daily Crashes',
          color='Red', source=df_crashes_by_month)

show(crashes_by_month_fig)
df_crashes_by_month

,Month,Daily Crashes
0,6,404
1,7,506
2,8,397
3,9,450
4,10,618
5,11,944
6,12,1743


[10p] Geographical Development: visualize the sales volume (as above) and the average 
rating per country in a geographical setting (using the geopandas package, see more 
information below) , for example the number of customers per country over time. The goal is 
again to give management as much geographic insight as possible.

In [12]:
shapefile = 'data/ne_110m_admin_0_countries.shp'
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
gdf.columns = ['country', 'country_code', 'geometry']
gdf = gdf.drop(gdf.index[159])

df_sales_monthly = df_sales.groupby(['Month', 'Country of Buyer'])[['Amount (Merchant Currency)']].sum().reset_index()
df_ratings_monthly = df_ratings.groupby(['Month', 'Country'])[['Total Average Rating']].mean().reset_index()
# df_sales_monthly['Month'] = df_sales_monthly['Month'].astype(int)

def convert_country_code(alpha_2):
    try:
        return pycountry.countries.get(alpha_2=alpha_2).alpha_3
    except AttributeError:
        return None  # Handle missing or incorrect country codes

df_sales_monthly['country_code'] = df_sales_monthly['Country of Buyer'].apply(convert_country_code)
df_ratings_monthly['country_code'] = df_ratings_monthly['Country'].apply(convert_country_code)

def get_dataset(month):
    """Filter data for a specific month and merge with GeoDataFrame"""
    filtered_df = df_sales_monthly[df_sales_monthly['Month'] == month]
    merged = gdf.merge(filtered_df, on="country_code", how="left")
    merged['Amount (Merchant Currency)'] = merged['Amount (Merchant Currency)'].fillna(0)
    return merged
    
def get_geodatasource(gdf):    
    """Convert GeoDataFrame to Bokeh-compatible GeoJSONDataSource"""
    json_data = json.dumps(json.loads(gdf.to_json()))
    return GeoJSONDataSource(geojson=json_data)

# initial_month = df_sales_monthly['Month'].min()
# merged_gdf = get_dataset(initial_month)

def bokeh_plot_map(gdf, column=None, title='Choropleth Map'):
    """Plot choropleth map using Bokeh"""
    
    geosource = get_geodatasource(gdf)
    palette = brewer['OrRd'][8][::-1]  # Reverse palette for correct color order
    vals = gdf[column].dropna()  # Drop NaN values to avoid color mapping issues

    # Create a color mapper
    color_mapper = LinearColorMapper(palette=palette, low=vals.min(), high=vals.max())

    # Create color bar
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=20,
                         location=(0,0), orientation='horizontal')

    tools = 'wheel_zoom,pan,reset'
    p = figure(title=title, height=500, width=850, toolbar_location='right', tools=tools)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    # Add patches (polygons) to the figure
    p.patches('xs', 'ys', source=geosource, fill_alpha=1, line_width=0.5, line_color='black',  
              fill_color={'field': column, 'transform': color_mapper})

    # Add color bar to the figure
    p.add_layout(color_bar, 'below')

    return p


df_sales_monthly['country_code'] = df_sales_monthly['Country of Buyer'].apply(convert_country_code)
df_ratings_monthly['country_code'] = df_ratings_monthly['Country'].apply(convert_country_code)

merged_gdf_sales = gdf.merge(df_sales_monthly, on="country_code", how="left")
merged_gdf_ratings = gdf.merge(df_ratings_monthly, on="country_code", how="left")

bokeh_map_sales = bokeh_plot_map(merged_gdf_sales, column='Amount (Merchant Currency)', title="Global App Ratings")
bokeh_map_ratings = bokeh_plot_map(merged_gdf_ratings, column='Total Average Rating', title="Global App Ratings")

show(bokeh_map_sales)
show(bokeh_map_ratings)

DataSourceError: data/ne_110m_admin_0_countries.shp: No such file or directory

Put it all together

In [ ]:
monthly_sales_panel = TabPanel(child=monthly_sales_fig, title="Monthly Sales")
sales_by_country_panel = TabPanel(child=sales_by_country_fig, title="Total Sales per Country")
sales_by_sku_panel = TabPanel(child=sales_by_sku_fig, title="Sales per SKU")

sales_tabs = Tabs(tabs=[monthly_sales_panel, sales_by_sku_panel, sales_by_country_panel])

crashes_panel = TabPanel(child=crashes_by_month_fig, title="Crashed per Month")
rating_panel = TabPanel(child=ratings_by_month_fig, title='AVG Rating per Month')

tabs = Tabs(tabs=[crashes_panel, rating_panel])

dashboard_grid = column(
                        gridplot([[sales_tabs, tabs]], merge_tools=False)
                        )

show(dashboard_grid)